In [31]:
import chart_studio
username = 'Rlee00'
api_key = 'h4FpnLQqMRenUd7hRb95'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

import chart_studio.plotly as py
import chart_studio.tools as tls

In [32]:
Year = [2018,2019,2020]
Tweets = [1,2,3]
Score = [0,-1,1]

In [33]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

trial = px.scatter_3d(x=Year,y=Tweets,z=Score)
trial

In [36]:
py.plot(trial, filename = 'trial.html', auto_open=True)

'https://plotly.com/~Rlee00/10/'

In [ ]:
import requests
import pandas as pd
import json
import ast
import yaml

Group_0_Follower = pd.read_csv("Group_0_Follower.csv",sep=",")
Group_0_Following = pd.read_csv("Group_0_Following.csv",sep=",")
result_last_update = pd.read_csv("result_last_update.csv",sep=",")
user_id_Group_0 = pd.read_csv("user_id_split_80/data/user_id_Group_0.csv",sep=",")
user_tweets = pd.read_csv("user_tweets.csv",sep=",")


In [10]:
user_tweets

,id,tweet cotents
0,547764853,RT @badboygargar: Day 17: morale is weird http...
1,547764853,RT @ArkhamVideos: What happens when you combin...
2,547764853,I want to #TravelWithPride to a @Lions road ga...
3,547764853,"""I want to #TravelWithPride to a @Lions game i..."
4,547764853,RT @sm_jamal11: HOMECOMING THEME IS PIXAR ‼️ V...
...,...,...
1107181,1517434428,@BasakGkbyrak Günün her saati itinayla :ADF:...
1107182,1517434428,İnsanım ama normal değilim :ASDF
1107183,1517434428,Bu laf sokma çabaların niye canısı az anlatsan...
1107184,1517434428,Bendeki hayal kırıklıkları tavan :(((((((((


In [11]:
def combine_lang_data(documents, with_languages):
    langs = pd.DataFrame(with_languages["documents"])
    lang_iso = [x.get("iso6391Name")
                for d in langs.detectedLanguages if d for x in d]
    data_only = documents["documents"]
    tweet_data = pd.DataFrame(data_only)
    tweet_data.insert(2, "language", lang_iso, True)
    json_lines = tweet_data.to_json(orient="records")
    return json_lines

In [12]:
def add_document_format(json_lines):
    docu_format = '"' + "documents" + '"'
    json_docu_format = "{}:{}".format(docu_format, json_lines)
    docu_align = "{" + json_docu_format + "}"
    jd_align = json.dumps(docu_align)
    jl_align = json.loads(jd_align)
    return ast.literal_eval(jl_align)

In [13]:
def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(
        sentiment_url, headers=headers, json=document_format)
    return response.json()

In [14]:
def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments["documents"])
    return sentiment_df["score"].mean()

In [15]:
def week_logic(week_score):
    if week_score > 0.75 or week_score == 0.75:
        print("You had a positive week")
    elif week_score > 0.45 or week_score == 0.45:
        print("You had a neutral week")
    else:
        print("You had a negative week, I hope it gets better")

In [16]:
def main():
    url = create_twitter_url()
    data = process_yaml()
    bearer_token = create_bearer_token(data)
    res_json = twitter_auth_and_connect(bearer_token, url)
    documents = lang_data_shape(res_json)
    language_api_url, sentiment_url, subscription_key = connect_to_azure(data)
    headers = azure_header(subscription_key)
    with_languages = generate_languages(headers, language_api_url, documents)
    json_lines = combine_lang_data(documents, with_languages)
    document_format = add_document_format(json_lines)
    sentiments = sentiment_scores(headers, sentiment_url, document_format)
    week_score = mean_score(sentiments)
    print(week_score)
    week_logic(week_score)